In [1]:
import pandas as pd 
import numpy as np
import scipy.stats

In [2]:
df=pd.read_excel('C:\\Users\\nafissaad\\Downloads\\Spreadsheets-20231017T124722Z-001\\Spreadsheets\\NEDL_Black_Litterman.xlsx',index_col='Date')[['JNJ','JPM','ORCL','VLO','WMT','SPX']]
df

,JNJ,JPM,ORCL,VLO,WMT,SPX
Date,,,,,,
2011-12-30,49.120182,25.086231,22.122772,13.154429,47.016327,1257.599976
2012-01-03,49.344910,26.391470,22.303892,13.079441,47.464783,1277.060059
2012-01-04,49.045303,26.558643,22.433270,12.760736,46.976990,1277.300049
2012-01-05,48.985371,27.113382,22.933504,12.379538,46.748825,1281.060059
2012-01-06,48.558437,26.870209,23.226753,12.729487,46.418404,1277.810059
...,...,...,...,...,...,...
2021-12-27,168.587799,157.217621,89.208603,72.301956,140.217682,4791.189941
2021-12-28,169.263474,157.694748,88.321892,73.656441,142.229904,4786.350098
2021-12-29,170.455811,157.615234,87.883522,73.804749,142.160172,4793.060059


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2518 entries, 2011-12-30 to 2021-12-31
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   JNJ     2518 non-null   float64
 1   JPM     2518 non-null   float64
 2   ORCL    2518 non-null   float64
 3   VLO     2518 non-null   float64
 4   WMT     2518 non-null   float64
 5   SPX     2518 non-null   float64
dtypes: float64(6)
memory usage: 137.7 KB


In [4]:
returns=df.pct_change().dropna()
returns

,JNJ,JPM,ORCL,VLO,WMT,SPX
Date,,,,,,
2012-01-03,0.004575,0.052030,0.008187,-0.005701,0.009538,0.015474
2012-01-04,-0.006072,0.006334,0.005801,-0.024367,-0.010277,0.000188
2012-01-05,-0.001222,0.020887,0.022299,-0.029873,-0.004857,0.002944
2012-01-06,-0.008716,-0.008969,0.012787,0.028268,-0.007068,-0.002537
2012-01-09,0.001542,-0.001697,0.005955,-0.037310,0.003051,0.002262
...,...,...,...,...,...,...
2021-12-27,0.008440,0.005723,-0.002006,0.019518,0.009105,0.013839
2021-12-28,0.004008,0.003035,-0.009940,0.018734,0.014351,-0.001010
2021-12-29,0.007044,-0.000504,-0.004963,0.002014,-0.000490,0.001402


In [5]:
matrix=pd.DataFrame(columns=['JNJ','JPM','ORCL','VLO','WMT','SPX'],index=['Return','Beta','CAPM'])
matrix

,JNJ,JPM,ORCL,VLO,WMT,SPX
Return,NaN,NaN,NaN,NaN,NaN,NaN
Beta,NaN,NaN,NaN,NaN,NaN,NaN
CAPM,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from scipy.stats import linregress
linregress((returns['JNJ']),np.array(returns['SPX']))[0]

0.6082937748433147

In [7]:
for i in matrix.columns:
    matrix.loc['Return',i]=(df[i][-1]/df[i][0])**(1/10)-1
    matrix.loc['Beta',i]=np.polyfit(np.array(returns['SPX']),np.array(returns[i]),1)[0]
    matrix.loc['CAPM',i]=0.0152+np.polyfit(np.array(returns['SPX']),np.array(returns[i]),1)[0]*(((df['SPX'][-1]/df['SPX'][0])**(1/10)-1)-0.0152)
matrix

,JNJ,JPM,ORCL,VLO,WMT,SPX
Return,0.132175,0.201597,0.146599,0.188962,0.11854,0.142517
Beta,0.663644,1.219839,0.976441,1.334046,0.542783,1.0
CAPM,0.099693,0.170507,0.139518,0.185047,0.084306,0.142517


In [8]:
returns[returns.columns[:-1]][:-1].cov()*252

,JNJ,JPM,ORCL,VLO,WMT
JNJ,0.029182,0.019779,0.018476,0.019729,0.013267
JPM,0.019779,0.071190,0.032475,0.056744,0.014095
ORCL,0.018476,0.032475,0.060575,0.033233,0.016309
VLO,0.019729,0.056744,0.033233,0.149425,0.013230
WMT,0.013267,0.014095,0.016309,0.013230,0.037791


In [9]:
#inverse Covariance
inverse=pd.DataFrame(columns=['JNJ','JPM','ORCL','VLO','WMT'],index=['JNJ','JPM','ORCL','VLO','WMT'])
inverse[['JNJ','JPM','ORCL','VLO','WMT']]=np.linalg.inv(returns[returns.columns[:-1]][:-1].cov()*252)
inverse

,JNJ,JPM,ORCL,VLO,WMT
JNJ,49.865327,-7.228035,-7.727387,-1.139492,-11.076915
JPM,-7.228035,25.012061,-7.087238,-6.849653,-1.334571
ORCL,-7.727387,-7.087238,24.710300,-1.356212,-4.832932
VLO,-1.139492,-6.849653,-1.356212,9.733869,0.132358
WMT,-11.076915,-1.334571,-4.832932,0.132358,32.887537


In [10]:
vectors=pd.DataFrame(columns=['JNJ','JPM','ORCL','VLO','WMT'],index=['e','r','h','g','market cap'])
vectors.loc['e']=1
vectors.loc['r']=matrix.loc['CAPM'][:-1]
vectors.loc['h']=np.array(vectors.loc['e']).dot(np.array(inverse))
vectors.loc['g']=np.array(vectors.loc['r']).dot(np.array(inverse))
vectors.loc['market cap']=[449.79,467.58,232.89,30.74,401.35]
vectors

,JNJ,JPM,ORCL,VLO,WMT
e,1,1,1,1,1
r,0.099693,0.170507,0.139518,0.185047,0.084306
h,22.6935,2.512565,3.706531,0.52087,15.775478
g,1.515997,1.17532,0.81033,0.341656,0.790972
market cap,449.79,467.58,232.89,30.74,401.35


In [11]:
alpha=np.array(vectors.loc['e']).dot(np.array(vectors.loc['h']))
beta=np.array(vectors.loc['e']).dot(np.array(vectors.loc['g']))
gamma=np.array(vectors.loc['r']).dot(np.array(vectors.loc['g']))
delta=alpha*gamma-beta**2
tangency=(gamma-beta*0.0152)/(beta-alpha*0.0152)#0.0152 risk free rate
lambdas=(gamma-beta*tangency)/delta
mu=(alpha*tangency-beta)/delta
alpha,beta,gamma,delta,tangency,lambdas,mu

(45.20894375964637,
 4.634275619997495,
 0.5944961757747906,
 5.400033653824185,
 0.13276968648394435,
 -0.0038509288470335674,
 0.25335058204168154)

In [12]:
sum(x * y for x, y in zip(vectors.loc['e'],vectors.loc['g']))

4.634275619997495

In [13]:
tangency=(gamma-beta*0.0152)/(beta-alpha*0.0152)#0.0152 risk free rate
tangency#return

0.13276968648394435

In [14]:
portfolio=lambdas*np.array(vectors.loc['h'])+mu*np.array(vectors.loc['g'])
portfolio

array([0.29668776717146517, 0.288092256980052, 0.19102406309617861,
       0.08455290212751938, 0.1396430106247844], dtype=object)

In [15]:
sum(lambdas*np.array(vectors.loc['h'])+mu*np.array(vectors.loc['g']))

0.9999999999999996

In [16]:
value_weights=np.array(vectors.loc['market cap'])/np.sum(vectors.loc['market cap'])
value_weights

array([0.2842544316996872, 0.29549720352640063, 0.14717982747179828,
       0.019426801908553733, 0.2536417353935602], dtype=object)

In [17]:
deviation=np.sum(np.square(portfolio-value_weights))
deviation
#we can optimize the function and make the deviation close to zero

0.01936885624043209

In [18]:
#Black-littreman model suggest us a portfolio that maximize expected return and penalize the risk, make a equilibrium.